In [2]:
from pyquery import PyQuery as pq
import urllib
from lxml import html
from lxml.cssselect import CSSSelector

In [3]:
# from lxml import etree
# from io import StringIO, BytesIO# broken_html = "<html><head><title>test<body><h1>page title</h3> <div><div id='divID' class='content'></div></div>"
# parser = etree.HTMLParser()
# tree   = etree.parse(BytesIO(broken_html), parser)
# result = etree.tostring(tree.getroot(),pretty_print=True, method="html")

In [36]:
def getFam(ID):
    url='http://pfam.xfam.org/protein/'+ID
    result = urllib.urlopen(url).read()
    h= html.fromstring(result)
    selector = CSSSelector("a[href^='http://pfam.xfam.org/family/']")
    families = [e.text_content() for e in selector(h)]
    return families

def next_col(h,text,index=-1,nxt=2):
    selector = CSSSelector("td:contains('{}')".format(text))
    result=selector(h)
    if(not result or len(result)==0):
        return ""
    elem=result[index]
    for i in range(nxt):
        elem=elem.getnext()
        
    return elem.text_content()
    
    
def down_col(h,text,index=-1,nxt=2):
    selector = CSSSelector("td:contains('{}')".format(text))
    result=selector(h)
    if(not result or len(result)==0):
        return ""
    elem=result[index]
    
    elem=elem.getparent().getnext().find('td')
    
    for i in range(nxt):
        elem=elem.getnext()
        
    return elem.text_content()
    

def getLoc(ID):
    url='https://www.bluedart.com/servlet/RoutingServlet?handler=tnt&action=awbquery&awb=awb&numbers='+str(ID)
    result = urllib.urlopen(url).read()
    h= html.fromstring(result)
    
    delivered = next_col(h,"SHIPMENT DELIVERED")    
    delayed= next_col(h,"Network Delay")
    from_city =next_col(h,"From",1)
    to_city =down_col(h,"From",1)
    return to_city
    if(delivered!=""):
        delivered_date = next_col(h,"Date of Delivery")
        delivered_time = next_col(h,"Time of Delivery")
        return "delivered",delivered_date,delivered_time
    elif (delayed!=""):
        exp_date = next_col(h,"Expected Date of Delivery")
        return "delayed",exp_date
    else:
        exp_date = next_col(h,"Expected Date of Delivery")
        if(exp_date==""):
            return "N/A - check manually"
        return exp_date


In [37]:
IDs = ['50450887594','50450887653','50450887093','50450887863','50450887314','50450887465','50450887690','50450887815','50450887852','50450887443','50450887712','50450887266','50450887303','50450887373','50450887476','50450887222','50450886721','50450886743','50450886916','50450887664','50450887351','50450887572','50450887804','50450886780','50450887255','50450887270','50450887292','50450887723','50450886894','50450887126','50450887583','50450887734','50450886920','50450887012','50450887362','50450887045','50450887675','50450887115','50450886673','50450887491','50450887163','50450887340','50450887561','50450887513','50450886986','50450887233','50450886975','50450886905','50450887771','50450887336','50450887056','50450887395','50450887034','50450886706','50450887421','50450887432','50450886662','50450886684','50450887830','50450887546','50450886710','50450887410','50450886990','50450887841','50450887023','DUMMY','50450886953','50450887605','50450887535','50450887480','50450887060','50450886765','50450887071','50450886754','50450887141','50450887756','50450887211','50450887616','50450887620','50450887631','50450887701','50450886732','50450887244','50450887760','50450887524','50450887001','50450887281','50450886931','50450886942','50450887885','50450887082','50450887325','50450887406','50450887745','50450887826','50450887874','50450887454','50450887642','50450886883','50450887550','50450887686','50450887384','50450887152','50450887782','50450887130','50450887502','50450886695','50450886964','50450887793']
locs=[]
for ID in IDs:
    loc = getLoc(ID)
    print(ID,loc)
    locs.append(loc)

('50450887594', 'Khargone ')
('50450887653', 'Raipur ')
('50450887093', 'Chhattarpur ')
('50450887863', 'Bilaspur ')
('50450887314', 'Chhindwara ')
('50450887465', 'Bhopal ')
('50450887690', 'Gwalior ')
('50450887815', 'Indore ')
('50450887852', 'Indore ')
('50450887443', 'Jabalpur ')
('50450887712', 'Raipur ')
('50450887266', 'Bhopal ')
('50450887303', 'Bhilai ')
('50450887373', 'Darbhanga ')
('50450887476', 'Bokaro ')
('50450887222', 'Gaya ')
('50450886721', 'Ranchi ')
('50450886743', 'Tufangangj ')
('50450886916', 'Puri ')
('50450887664', 'Muzaffarpur ')
('50450887351', 'Patna ')
('50450887572', 'Bhagalpur ')
('50450887804', 'Purnia ')
('50450886780', 'Berhampur ')
('50450887255', 'Patna ')
('50450887270', 'Patna ')
('50450887292', 'Berhampur ')
('50450887723', 'New Delhi ')
('50450886894', 'New Delhi ')
('50450887126', 'Chandigarh ')
('50450887583', 'Faridabad ')
('50450887734', 'Rewari ')
('50450886920', 'Gorakhpur ')
('50450887012', 'Patiala ')
('50450887362', 'Raebareli ')
('504

In [38]:
import copy

len(IDs) == len(locs)

True

In [39]:
import pyperclip
pyperclip.copy(str(locs))